<h2>Importe</h2>

In [2]:
import pickle

import numpy
import pandas as pd
import numpy as np
import os
import src.data as data
import torch
import os
from sklearn import metrics
import matplotlib.pyplot as plt
import math


<h2>Methoden</h2>

In [1]:
#Straight up stolen from main.py
def calc_roc_auc(gts, probs):
    try:
        auc = metrics.roc_auc_score(gts, probs)
        if np.isnan(auc):
            auc = 0

        return auc
    except:
        print("except")
        return 0


In [3]:
def map_value(value):
    if value >= 0.5:
        return 1
    else:
        return 0

<h2>Laden der gespeicherten Daten und der Modelle</h2>

In [4]:
data_list = []
with open(r"C:\Users\ReneJ\Desktop\UnityStuff\patway-net\data_plot\test_data", "rb") as input:
    while True:
        try:
            x = pickle.load(input)
        except EOFError:
            break
        data_list.append(x)



In [7]:
len(data_list)

5

In [5]:
model_list = []
dir = r"C:\Users\ReneJ\Desktop\UnityStuff\patway-net\model"
for file in os.listdir(dir):
    filename = os.fsdecode(file)
    if "_15" in filename:
        model_list.append(torch.load(dir + "\\" + file))


In [9]:
len(model_list)

5

In [6]:
lowest_len = math.inf
for ds in data_list:
    if len(ds["x_test_seq"]) < lowest_len:
        lowest_len = len(ds["x_test_seq"])


In [11]:
lowest_len

183

In [59]:
(data_list[0]["x_test_seq"][5:10])

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

    

In [15]:
y_values = []

n = 0
for model, dataset in zip(model_list,data_list):
    values = {"fold" : dataset["fold"], "data" : []}
    print(n)
    n += 1
    i = 5
    pre_i = 0
    seq = torch.from_numpy(dataset["x_test_seq"])
    stat = torch.from_numpy(dataset["x_test_stat"])
    label = torch.from_numpy(dataset["label"])
    timestamps = seq.shape[1]
    for t in range(1, timestamps):
        pre_t = 0
        model.eval()
        with torch.no_grad():
            seq_ = seq[:,pre_t:t,:]

            prediction = torch.sigmoid(model.forward(seq[i][0:5], stat[0:5]))

            prediction = torch.sigmoid(model.forward(seq, stat))
            prediction = [map_value([0][0]) for pred in prediction]
            print(prediction)
            #values["data"].append(calc_roc_auc(dataset["label"], prediction))
            fpr, tpr, x = metrics.roc_curve(dataset["label"][pre_i : i], prediction)
            values["data"].append(metrics.auc(fpr, tpr))

        pre_t = t
    y_values.append(values)

0


KeyboardInterrupt: 

In [134]:
torch.from_numpy(dataset["x_test_stat"])[[0,2,4]]

tensor([[1.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.6111,
         1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.0000,
         0.0000, 1.0000, 0.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.6111,
         1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.0000,
         0.0000, 1.0000, 0.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.6111,
         1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.0000,
         0.0000, 1.0000, 0.0000, 1.0000, 1.0000]], dtype=torch.float64)

In [7]:
def getPrefixLength(seq, sample):
    counter = 0
    rowCounter = -1
    lastRow = None
    actualPrefixRows = 0
    for matrix in seq[sample,:,:]:
        #print(matrix)
        rowCounter += 1

        notZero = False

        for x in matrix:
            x = float(x)
            if x != 0:
                notZero = True
                break

        if notZero:
            lastRow = matrix
            actualPrefixRows = rowCounter

    if lastRow != None:
        for entry in lastRow:
            if entry == 0:
                counter += 1

            if entry != 0:
                counter += 1
                break

    #print(actualPrefixRows)
    #print(counter)
    #print(rowCounter)
    #print(lastRow)

    return counter + (actualPrefixRows * len(lastRow))

In [24]:
getPrefixLength((torch.from_numpy(data_list[0]["x_test_seq"])), 23)
torch.from_numpy(data_list[0]["x_test_seq"])[51,:,:]
#torch.from_numpy(data_list[0]["x_test_seq"])[0]

tensor([[0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.5404, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.5404, 0.2261, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.00

In [11]:
def getPrefixDictionary(seq):
    samples = seq.shape[0]
    print(samples)

    mapList = []
    uniquePrefixSizes = []
    for t in range(0, samples):
        indexPrefixSizeMap = {"index" : t, "prefixSize" : getPrefixLength(seq, t)}
        #print("sample " + str(t) + ": " + str(getPrefixLength(seq, t)))
        mapList.append(indexPrefixSizeMap)
        uniquePrefixSizes.append(getPrefixLength(seq, t))

    uniquePrefixSizes = set(uniquePrefixSizes)
    #print(uniquePrefixSizes)

    prefixDictionaryList = []

    for prefixSize in uniquePrefixSizes:
        prefixDictionary = {}
        prefixDictionary["prefixSize"] = prefixSize
        prefixDictionary["indizes"] = []
        for entry in mapList:
            if entry["prefixSize"] == prefixSize:
                prefixDictionary["indizes"].append(entry["index"])
        prefixDictionaryList.append(prefixDictionary)

    #print(prefixDictionary)
    return prefixDictionaryList


In [12]:
getPrefixDictionary((torch.from_numpy(data_list[0]["x_test_seq"])))

183


[{'prefixSize': 129, 'indizes': [8, 24, 41, 55, 70, 102, 122, 144, 164, 177]},
 {'prefixSize': 257, 'indizes': [32]},
 {'prefixSize': 4,
  'indizes': [0,
   16,
   33,
   44,
   47,
   62,
   72,
   80,
   88,
   94,
   106,
   114,
   125,
   133,
   136,
   151,
   156,
   169]},
 {'prefixSize': 145, 'indizes': [9, 25, 42, 56, 71, 103, 123, 145, 165, 178]},
 {'prefixSize': 21,
  'indizes': [1,
   17,
   34,
   45,
   48,
   63,
   73,
   81,
   89,
   95,
   107,
   115,
   126,
   134,
   137,
   152,
   157,
   170]},
 {'prefixSize': 161, 'indizes': [10, 26, 43, 57, 104, 124, 146, 166, 179]},
 {'prefixSize': 34, 'indizes': [74]},
 {'prefixSize': 38,
  'indizes': [2,
   18,
   35,
   46,
   49,
   64,
   82,
   90,
   96,
   108,
   116,
   127,
   135,
   138,
   153,
   158,
   171]},
 {'prefixSize': 177, 'indizes': [11, 27, 58, 105, 147, 167, 180]},
 {'prefixSize': 50, 'indizes': [50, 75, 109, 139]},
 {'prefixSize': 51, 'indizes': [19, 36]},
 {'prefixSize': 49, 'indizes': [65, 91

In [14]:
getPrefixLength((torch.from_numpy(data_list[0]["x_test_seq"])), 10)

161

In [29]:
result = []

for model, dataset in zip(model_list,data_list):
    values = {"model" : dataset["fold"], "data" : []}

    seq = torch.from_numpy(dataset["x_test_seq"])
    stat = torch.from_numpy(dataset["x_test_stat"])
    label = torch.from_numpy(dataset["label"])

    prefixLengthDictList = getPrefixDictionary(seq)

    for prefixLengthDict in prefixLengthDictList:
        performancePrefixDict = {"PrefixLength" : prefixLengthDict["prefixSize"]}

        prefixSeq = seq[prefixLengthDict["indizes"],:,:]
        prefixStat = stat[prefixLengthDict["indizes"],:]
        prefixLabel = label[prefixLengthDict["indizes"]]

        model.eval()
        with torch.no_grad():

            prediction = torch.sigmoid(model.forward(prefixSeq, prefixStat))
            prediction = [map_value([0][0]) for pred in prediction]
            #print((prediction))
            #print((prefixLabel))

            performancePrefixDict["AUC"] = calc_roc_auc(prefixLabel, prediction)

            #rocauc = metrics.roc_auc_score(prefixLabel, prediction)
            #print(rocauc)
            #performancePrefixDict["AUC"] = rocauc

            values["data"].append(performancePrefixDict)

    result.append(values)


183
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
194
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
185
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
197
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
191
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except
except


In [34]:

for dict in result:
    x_axis = dict["data"]["PrefixLength"]
    plt.plot(x_axis, dict["data"]["AUC"], label = dict["model"])

plt.legend()
plt.show()

TypeError: list indices must be integers or slices, not str